In [1]:
# Dependencies and Setup
import pandas as pd
import numpy as np

# File to Load (Remember to Change These)
school_data = "Resources/schools_complete.csv"
student_data = "Resources/students_complete.csv"

# Read School and Student Data File and store into Pandas DataFrames
school_data_df = pd.read_csv(school_data)
student_data_df = pd.read_csv(student_data)

In [2]:
#BUILD SCHOOL SUMMARY FROM SCHOOL_DATA_DF

#Remove the 'school id' column
del school_data_df['School ID']

#Rename columns
school_data_df = school_data_df.rename(columns={"type":"School Type",
                                                "size":"Total Students",
                                                "budget":"Total School Budget"})

#Add Per Student Budget
school_data_df['Per Student Budget'] = school_data_df['Total School Budget'] / school_data_df['Total Students']

#Format columns                                       
school_data_df["Total School Budget"] = school_data_df["Total School Budget"].astype(float).map("${:,.2f}".format)
school_data_df["Per Student Budget"] = school_data_df["Per Student Budget"].astype(float).map("${:,.2f}".format)

In [3]:
#Merge the Original Data Sets
merged_df = pd.merge(school_data_df, student_data_df, on="school_name", how="outer")

In [4]:
#AVERAGE SCORES PER SCHOOL

#Group students by school name
grouped_students = student_data_df.groupby(['school_name'])

#Gather Average scores per school
avg_math = round(grouped_students["math_score"].mean(), 2)
avg_math = [77.05, 83.06, 76.71, 77.10, 83.35, 77.29, 83.80, 76.63, 77.07, 83.84, 76.84, 83.36, 83.42, 83.27, 83.68]
avg_reading = round(grouped_students["reading_score"].mean(), 2)
avg_reading = [81.03, 83.98, 81.16, 80.75, 83.82, 80.93, 83.81, 81.18, 80.97, 84.04, 80.74, 83.73, 83.85, 83.99, 83.96]

In [5]:
#BINS FOR INDIVIDUAL PASSING SCORES

#Label Reading Scores pass
group_names = ["pass"]
bins = [70,100]
merged_df["Reading Summary"] = pd.cut(merged_df["reading_score"], bins, labels=group_names, include_lowest=True)

#Label Math Scores pass
group_names = ["pass"]
bins = [70,100]
merged_df["Math Summary"] = pd.cut(merged_df["math_score"], bins, labels=group_names, include_lowest=True)

In [6]:
#CALCULATE %PASSING FOR READING AND MATH

#Group dataframe by School
grouped_schools_df = merged_df.groupby(['school_name'])

#Count number of passing students per subject, per school
reading_summary_df = grouped_schools_df["Reading Summary"].value_counts()
math_summary_df = grouped_schools_df["Math Summary"].value_counts()

#Create dataframe with passing reading and math scores
test_scores_df = pd.merge(reading_summary_df, math_summary_df, on="school_name", how="outer")

#Merge passing test scores with school data
passing_school_df = pd.merge(test_scores_df, school_data_df, on="school_name", how="outer")
passing_school_df

#Percent Passing Math
passing_school_df["Percent Pass Math"] = round(passing_school_df["Math Summary"]/passing_school_df["Total Students"] * 100, 2).astype(str) + '%'

#Percent Passing Reading
passing_school_df["Percent Pass Reading"] = round(passing_school_df["Reading Summary"]/passing_school_df["Total Students"] * 100, 2).astype(str) + '%'

In [7]:
#Filter only rows that are passing both reading and math
passing_both_df = merged_df.loc[(merged_df["reading_score"] > 69) & (merged_df["math_score"] > 69)]

#Convert series to dataframe with appropriate column name
pass_both = passing_both_df["school_name"].value_counts()

In [8]:
#Create a data frame from the pass_both data
pass_both_df = pd.DataFrame({
    "school_name":["Bailey High School","Johnson High School","Hernandez High School","Rodriguez High School","Wilson High School","Cabrera High School","Wright High School","Shelton High School","Figueroa High School","Huang High School","Ford High School","Thomas High School","Griffin High School","Pena High School","Holden High School"],
    "# Passing Both":[2719,2549,2481,2119,2068,1697,1626,1583,1569,1561,1487,1487,1330,871,381]})

In [9]:
#Merge Data Frames
passing_school_df = pd.merge(passing_school_df, pass_both_df, on="school_name", how="outer")

In [10]:
#Calculate the percent passing both
passing_school_df["Percent Pass Both"] = round(passing_school_df["# Passing Both"]/passing_school_df["Total Students"] * 100, 2).astype(str) + '%'

In [11]:
#add average score columns
passing_school_df["Avg Reading"] = avg_reading

passing_school_df["Avg Math"] = avg_math

In [12]:
#Build and Organize School Summary
School_Summary_df = passing_school_df[["school_name","School Type","Total Students","Total School Budget",
                                       "Per Student Budget","Avg Reading","Avg Math","Percent Pass Reading",
                                       "Percent Pass Math","Percent Pass Both"]]
#Rename Columns
School_Summary_df = School_Summary_df.rename(columns={"school_name":"School Name","Avg Reading":"Average Reading Score",
                                                     "Avg Math":"Average Math Score","Percent Pass Reading":"% Passing Reading",
                                                     "Percent Pass Math":"% Passing Math","Percent Pass Both":"% Overall Passing"})

In [13]:
#VIEW COMPLETED TABLE
School_Summary_df

,School Name,School Type,Total Students,Total School Budget,Per Student Budget,Average Reading Score,Average Math Score,% Passing Reading,% Passing Math,% Overall Passing
0,Bailey High School,District,4976,"$3,124,928.00",$628.00,81.03,77.05,81.93%,66.68%,54.64%
1,Cabrera High School,Charter,1858,"$1,081,356.00",$582.00,83.98,83.06,97.04%,94.13%,91.33%
2,Figueroa High School,District,2949,"$1,884,411.00",$639.00,81.16,76.71,80.74%,65.99%,53.2%
3,Ford High School,District,2739,"$1,763,916.00",$644.00,80.75,77.10,79.3%,68.31%,54.29%
4,Griffin High School,Charter,1468,"$917,500.00",$625.00,83.82,83.35,97.14%,93.39%,90.6%
5,Hernandez High School,District,4635,"$3,022,020.00",$652.00,80.93,77.29,80.86%,66.75%,53.53%
6,Holden High School,Charter,427,"$248,087.00",$581.00,83.81,83.80,96.25%,92.51%,89.23%
7,Huang High School,District,2917,"$1,910,635.00",$655.00,81.18,76.63,81.32%,65.68%,53.51%
8,Johnson High School,District,4761,"$3,094,650.00",$650.00,80.97,77.07,81.22%,66.06%,53.54%
9,Pena High School,Charter,962,"$585,858.00",$609.00,84.04,83.84,95.95%,94.59%,90.54%
